In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="theme/sixty_north.css">

# `unittest.mock`
**Test how objects are used**

|                    |               Links                                           |
|--------------------|---------------------------------------------------------------|
| Main documentation | https://docs.python.org/3/library/unittest.mock.html          |
| Getting started    | https://docs.python.org/3/library/unittest.mock-examples.html |

## Two main aspects

### Mocking
Instances of *`Mock`* can be used in place of other objects, and they keep track of how they're used.

### Patching
The *`patch()`* functions help you (temporarily) replace existing objects with `Mock`s.

* Basic idea "It allows you to replace parts of your system under test with mock objects and make assertions about how they have been used."
* Mock class / MagicMock class 
                  "MagicMock is a subclass of Mock with default implementations of most of the magic methods."
    * Create attributes as they are accessed
    * Specify return values, limitations, etc.
    * record details of how they're used
                  * called, call_count, call-args, call_args_list, method_calls, mock-calls
    * Assert about how they're used
                  * various assertions
    * side_effect: TODO
                  * callable
                  * list
                  * exception class or instance
    * return value
    * name
    * spec: TODO
    * reset ,add_spec, configure_mock,
    * non-callable versions
    * child-parent relationship / mock as attributes of mocks
                  * naming vs. not naming mock attributes
    
* patch():
                  * class decorator
                  * function decorator
                  * context manager
                  * patching `TestCase` subclasses
* patch.object
* patch.dict: TODO
* autospeccing, create_autospec
* patch() and Mock() take arbitrary keywords
* mock_open

# `unittest.mock.Mock`
*Testable replacements for other objects*

* Creates attributes on access
* Simulates callable and non-callable objects
* Records how it is used

In [ ]:
from unittest.mock import call, Mock

# accessing mocks creates new mocks by default
m = Mock()
m
# m()
# m.foo
# m.foo()

## Return values for `Mock`s

You can specify a return value in two ways, with an initializer argument:

```python
m = Mock(return_value=...)
```
or by setting an attribute:
```python
m.return_value = ...
```

In [ ]:
m = Mock(return_value=42)
m()

In [ ]:
m.return_value = "'I don't know what I was expecting.' -- Michael Bloom"
m()

## Side-effects for `Mocks`
*An object determing how a `Mock` behaves when called*

Specify a side-effect via an initializer argument:
```python
m = Mock(side_effect=...)
```
or by setting an attribute:
```python
m.side_effect = ...
```

### Callable side-effects
*Return value of callable is return value of calling the mock*

In [ ]:
# Callable side-effects produce the Mock's return value
def add(x, y):
    return x + y

m = Mock(side_effect=add)
m(1, 2)
m("ambi", "dextrous")

### The side-effect must follow arity rules, etc.

In [ ]:
m(1,2,3)   

### Exception side-effects
*Exception instances and classes cause a throw when the mock is called*

In [ ]:
# An exception type creates a new instance for each call
m = Mock(side_effect=ValueError)
try:
    m()
except ValueError:
    print('exception raised!')

In [ ]:
# An exception instance uses the same instance for each call
m.side_effect = ValueError("oops!")
try:
    m()
except ValueError as e:
    assert e.args[0] == "oops!"

### Iterable side-effects
*The iterable's values are the consecutive return values of the mock*

In [ ]:
m = Mock(side_effect=[1,2,3])
m(), m(), m()

##  `Mock` remembers how it is used
*This is the core of how `Mock` can be tested*

* Determine if and/or how many times it was called
* See the arguments used when it is called

In [ ]:
m = Mock()
m()
"Called? {}. Call count = {}.".format(
    m.called,
    m.call_count)

###  `call_args` and `call_args_list` track the arguments to `Mock` calls

In [ ]:
# `call_args` get the *last* argument set. `call_args_list` gets all of them.
m = Mock()
m(1,2,3)
m("This", "that", last="the other")
m.call_args
# m.call_args_list

### `method_calls` tracks calls to methods (and methods on methods!) on the `Mock`

In [ ]:
m = Mock()
m() # This is *not* reported in `method_calls`
m.some_method()
m.what.about.demeter("?!")
m.method_calls

### `mock_calls` tracks everything

In [ ]:
m = Mock()
# `mock_calls` tracks...
m(1337)              # ...calls to the Mock
m.attr("")           # ...calls to attributes of the Mock
m.deep(1).linking(2) # ...Call to the results of calls to attributes 
m.mock_calls

# Assert on how a `Mock` is used
*`Mock` has methods for checking how it was accessed and called*

* Was it called?
* Was it called with specific arguments?
* Does it have a specific call sequence?

## Checking arguments

In [ ]:
m = Mock()

# Was there only one call with exactly these arguments?
m(beta='b', alpha='a')
m.assert_called_once_with(alpha='a', beta='b')

# Did the most recent call have these arguments?
m(1,2, opt=3)
m.assert_called_with(1,2, opt=3)


## Checking call chains

In [ ]:
m = Mock()
m('pelagic')
m('argosy')
m('sights')
m('land')

# Check a specific sequence of calls
m.assert_has_calls(call(s) for s in ['pelagic', 'argosy', 'sights', 'land'])

# Check if *any* call had these arguments
m.assert_any_call('argosy')